In [ ]:
import numpy as np
import pandas as pd
from collections import Counter 
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, roc_auc_score, make_scorer
from sklearn.model_selection import cross_val_score, cross_validate
from scipy.stats import uniform, loguniform, randint
import matplotlib.pylab as plt
from matplotlib.pyplot import figure
import datetime

# Prepare Data 

In [ ]:
def load_data():
    df_train = pd.read_csv('data/corona_tested_individuals_ver_006.english_cleaned.csv')
    df_train.drop('Unnamed: 0', axis=1, inplace=True)
    df_train['test_date'] = pd.to_datetime(df_train['test_date'], format='%Y-%m-%d')
    df_train = df_train.set_index('test_date')
    df_train = df_train.rename_axis(index=None, axis=1)
    df_train.astype(int)
    df_train_0322_0331 = df_train.truncate(before=pd.Timestamp('2020-03-22'), after=pd.Timestamp('2020-03-31'))
    df_test_0401_0407 = df_train.truncate(before=pd.Timestamp('2020-04-01'), after=pd.Timestamp('2020-04-07'))
    var_col = [c for c in df_train if c not in ['corona_result']]
    X_train = df_train_0322_0331.loc[:, var_col]
    y_train = df_train_0322_0331.loc[:, 'corona_result']
    X_test = df_test_0401_0407.loc[:, var_col]
    y_test = df_test_0401_0407.loc[:, 'corona_result']
    return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = load_data()

# Define Train & Evaluate Functions

In [5]:
def sens_func(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    return tp/(tp+fn)
def spec_func(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    return tn/(tn+fp)
sensitivity_scorer = make_scorer(sens_func)
specificity_scorer = make_scorer(spec_func)

In [6]:
def evalModel(clf, X_train, y_train, X_test, y_test):
    print('cross validating: {}'.format(clf))
    
    scoring = {'sensitivity': sensitivity_scorer,
               'specificity': specificity_scorer,
               'accuracy': 'accuracy',
               'precision': 'precision',
               'roc_auc': 'roc_auc'}
    scores = cross_validate(clf, X_train, y_train, scoring=scoring, cv=5, n_jobs=-1)
    sensitivity = round(scores['test_sensitivity'].mean()*100, 2)
    specificity = round(scores['test_specificity'].mean()*100, 2)
    accuracy = round(scores['test_accuracy'].mean()*100, 2)
    precision = round(scores['test_precision'].mean()*100, 2)
    ROC = round(scores['test_roc_auc'].mean()*100, 2)
    train_scores = [sensitivity, specificity, accuracy, precision, ROC]

    print('fitting...')
    clf.fit(X_train, y_train)
    print('predicting...')
    y_pred = clf.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm.ravel()
    sensitivity = round(tp/(tp+fn)*100, 2)
    specificity = round(tn/(tn+fp)*100, 2)
    accuracy = round((tp+tn)/(tn+fp+fn+tp)*100, 2)
    precision = round(tp/(tp+fp)*100, 2)
    probs = clf.predict_proba(X_test)
    prob = probs[:, 1]
    ROC = round(roc_auc_score(y_test, prob)*100, 2)
    test_scores = [sensitivity, specificity, accuracy, precision, ROC]
    
    # print('saving model...')
    # pickle.dump(clf, open('xgboost_1.6.1.pkl', 'wb'))
    return train_scores, test_scores, cm

# Define Model
- Using the best model we get from grid searching

In [6]:
model = XGBClassifier(n_estimators=400, learning_rate=0.01, n_jobs=-1, random_state=42)

# Create Orthogonal Array

In [5]:
cols = np.array(X_train.columns)
OA = [[1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 2],
       [1, 1, 1, 2, 2, 2, 2, 1],
       [1, 1, 1, 2, 2, 2, 2, 2],
       [1, 2, 2, 1, 1, 2, 2, 1],
       [1, 2, 2, 1, 1, 2, 2, 2],
       [1, 2, 2, 2, 2, 1, 1, 1],
       [1, 2, 2, 2, 2, 1, 1, 2],
       [2, 1, 2, 1, 2, 1, 2, 1],
       [2, 1, 2, 1, 2, 1, 2, 2],
       [2, 1, 2, 2, 1, 2, 1, 1],
       [2, 1, 2, 2, 1, 2, 1, 2],
       [2, 2, 1, 1, 2, 2, 1, 1],
       [2, 2, 1, 1, 2, 2, 1, 2],
       [2, 2, 1, 2, 1, 1, 2, 1],
       [2, 2, 1, 2, 1, 1, 2, 2]]

In [10]:
S1_acc = np.zeros(8)
S2_acc = np.zeros(8)
S1_roc = np.zeros(8)
S2_roc = np.zeros(8)
for l in OA:
    col_new = []
    for v, col in zip(l, cols):
        if v == 1:
            col_new.append(col)
    X_train_new = X_train[col_new]
    X_test_new = X_test[col_new]
    train_scores, test_scores, cm = evalModel(model, X_train_new, y_train, X_test_new, y_test)
    acc = round(train_scores[2], 2)
    roc = round(train_scores[4], 2)
    for i, v in enumerate(l):
        if v == 1:
            S1_acc[i] = S1_acc[i]+acc
            S1_roc[i] = S1_roc[i]+roc          
        else:
            S2_acc[i] = S2_acc[i]+acc
            S2_roc[i] = S2_roc[i]+roc 
    l.append(acc)
    l.append(roc)

cross validating: XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.01, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=400,
              n_jobs=-1, num_parallel_tree=1, predictor='auto', random_state=42,
              reg_alpha=0, reg_lambda=1, ...)
fitting...
predicting...
cross validating: XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=

/var/folders/_9/sr4c36qx7hbggrnnvqjvqwvc0000gn/T/ipykernel_21206/1032075700.py:26: RuntimeWarning: invalid value encountered in long_scalars
  precision = round(tp/(tp+fp)*100, 2)


fitting...
predicting...
cross validating: XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.01, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=400,
              n_jobs=-1, num_parallel_tree=1, predictor='auto', random_state=42,
              reg_alpha=0, reg_lambda=1, ...)
fitting...
predicting...
cross validating: XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
             

In [11]:
MED_acc = []
better_level_acc = []
MED_roc = []
better_level_roc = []
for i in range(len(S1_acc)):
    S1_acc[i] = round(S1_acc[i], 2)
    S2_acc[i] = round(S2_acc[i], 2)
    med_acc = round(S1_acc[i] - S2_acc[i], 2)
    better_level_acc.append(1 if med_acc>0 else 2)
    MED_acc.append(abs(med_acc))
    S1_roc[i] = round(S1_roc[i], 2)
    S2_roc[i] = round(S2_roc[i], 2)
    med_roc = round(S1_roc[i] - S2_roc[i], 2)
    better_level_roc.append(1 if med_roc>0 else 2)
    MED_roc.append(abs(med_roc))

- reference: https://www.geeksforgeeks.org/rank-elements-array/

In [12]:
# Python code to find rank of elements
def rankify_improved(A):
	
	# create rank vector
	R = [0 for i in range(len(A))]

	# Create an auxiliary array of tuples
	# Each tuple stores the data as well
	# as its index in A
	T = [(A[i], i) for i in range(len(A))]

	# T[][0] is the data and T[][1] is
	# the index of data in A

	# Sort T according to first element
	T.sort(key=lambda x: x[0])

	(rank, n, i) = (len(A), 1, 0)

	while i < len(A):
		j = i

		# Get no of elements with equal rank
		while j < len(A) - 1 and T[j][0] == T[j + 1][0]:
			j += 1
		n = j - i + 1

		for j in range(n):

			# For each equal element use formula
			# obtain index of T[i+j][0] in A
			idx = T[i+j][1]
			R[idx] = rank + (n - 1) * 0.5

		# Increment rank and i
		rank -= n
		i += n

	return R

In [13]:
rank_acc = rankify_improved(MED_acc)
rank_roc = rankify_improved(MED_roc)

In [14]:
d = OA + [S1_acc, S2_acc, MED_acc, rank_acc, better_level_acc] + [S1_roc, S2_roc, MED_roc, rank_roc, better_level_roc]
index = list(range(1, 17)) + ['ACC S1', 'ACC S2', 'ACC MED', 'ACC Rank', 'ACC Better level'] + ['ROC S1', 'ROC S2', 'ROC MED', 'ROC Rank', 'ROC Better level']
columns = list(cols) + ['ACC', 'ROC']
oa_table = pd.DataFrame(d, index= index, columns=columns)

In [15]:
oa_table

,cough,fever,sore_throat,shortness_of_breath,head_ache,age_60_and_above,gender,test_indication,ACC,ROC
1,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,95.81,93.64
2,1.00,1.00,1.00,1.00,1.00,1.00,1.00,2.00,94.51,87.13
3,1.00,1.00,1.00,2.00,2.00,2.00,2.00,1.00,94.44,91.03
4,1.00,1.00,1.00,2.00,2.00,2.00,2.00,2.00,92.77,82.49
5,1.00,2.00,2.00,1.00,1.00,2.00,2.00,1.00,95.31,88.52
6,1.00,2.00,2.00,1.00,1.00,2.00,2.00,2.00,93.56,77.76
7,1.00,2.00,2.00,2.00,2.00,1.00,1.00,1.00,93.95,87.02
8,1.00,2.00,2.00,2.00,2.00,1.00,1.00,2.00,90.67,72.96
9,2.00,1.00,2.00,1.00,2.00,1.00,2.00,1.00,94.33,87.95
10,2.00,1.00,2.00,1.00,2.00,1.00,2.00,2.00,92.00,75.39


In [16]:
oa_table.to_csv('csv/xgboost_oa.csv')

/Users/changcw/miniforge3/envs/myenv/lib/python3.8/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: a7064dbb7c144eaac791dbc910e82230f5aeff9d- is an invalid version and will not be supported in a future release
  warnings.warn(
/Users/changcw/miniforge3/envs/myenv/lib/python3.8/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: a7064dbb7c144eaac791dbc910e82230f5aeff9d- is an invalid version and will not be supported in a future release
  warnings.warn(
/Users/changcw/miniforge3/envs/myenv/lib/python3.8/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: a7064dbb7c144eaac791dbc910e82230f5aeff9d- is an invalid version and will not be supported in a future release
  warnings.warn(
/Users/changcw/miniforge3/envs/myenv/lib/python3.8/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: a7064dbb7c144eaac791dbc910e82230f5aeff9d- is an invalid version and will not be supported in a future relea